In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
import requests

from datetime import timedelta, date

import itertools

In [2]:
dates=[] #pasirinktų datų sąrašas nuskaitomiems duomenims

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2015, 3, 20) 
end_dt = date(2015, 3, 20)


for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%Y-%m-%d"))
    
dates

['2015-03-20']

In [3]:
dfs1 = [] #Lentelių sąrašas paros duomenų lentelei apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus sukurti
#dfs2 = [] Lentelių sąrašas paros gaisrų aprašymų lentelei sukurti

for dateurl in dates:
    url = 'http://www.vpgt.lt/vpgt/m/m_events_summary/wfiles/e_'+dateurl+'.htm'
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    
    data0=[] #duomenų sąrašas
    
    for tr in rows:
        cols = tr.find_all('td')
        for td in cols:
            text = td.find_all(text=True)
            for t in text:
                data0.append(t)

                
    #Paros duomenys apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
    data0 = [w.replace('2.\r\n  Gaisrų aprašymas', '2. Gaisrų aprašymas') for w in data0]
    data1 = data0[data0.index('Ekstremaliąsias situacijas'):data0.index('2. Gaisrų aprašymas')]
    data1 = [a for a in data1 if a != '\n' and a != '\xa0 ']
    if data1[140] == 'd) pagalba ':
        data1[140:143] = [''.join(data1[140:143])]
    
    df01 = pd.DataFrame(np.array(data1).reshape(22,14), columns =["Duomenys apie:", "2017 nuo metų pr.", "2018 nuo metų pr.", "Iš viso per parą", "Vilniaus apskritis", "Kauno apskritis", "Klaipėdos apskritis", "Šiaulių apskritis", "Panevėžio apskritis", "Alytaus apskritis", "Utenos apskritis", "Marijampolės apskritis", "Tauragės apskritis", "Telšių apskritis"])
    df01 = df01.drop(df01.columns[1:4], axis=1)
    df01 = df01.T.drop([2,6,13], axis=1)
    df01 = df01.rename(columns=df01.iloc[0]).drop(['Duomenys apie:']).reset_index()
    df01['Data'] = dateurl
    
    dfs1.append(df01)
    
    
    ############
    #Paros gaisrų aprašymų duomenys
    #data2 = data0[data0.index('2.\r\n  Gaisrų aprašymas'):data0.index('XXXXXXXXXXXXXXXX')]
    #df02
    #dfs2.append(df02)
    ############

    
# paros duomenų lentelė df1 apie ekstrem. situacijas ir įvykius, gaisrus bei gelbėjimo darbus
df1 = pd.concat(dfs1, ignore_index=True)
df1['Žuvę ne gaisre'], df1['Traumuoti ne gaisre'] = df1['Žuvę/traumuoti ne gaisre'].str.split('/',1).str
df1 = df1.drop(['Žuvę/traumuoti ne gaisre'], axis=1)
df1 = df1.rename(index=str, columns={"index": "Apskr", "Ekstremaliąsias situacijas": "EkstSitu", 
                                    "Ekstremaliuosius įvykius": "EkstIvyk", "GAISRUS:": "GAISRAI", 
                                    "a) Gaisruose žuvusius": "Zuve_gaisr", "b) traumuotus asmenis": "Trau_gaisr", 
                                    "a) transporto avarijose": "GD_tr_av", "b) vandenyje": "GD_vand", 
                                    "c) buityje": "GD_buit", "d) pagalba spec. tarnyboms": "GD_sp_trn", 
                                    "e) chemija ir radiacija": "GD_ch_rad", "f) kitus": "GD_kiti", 
                                    "a) gaisruose": "Isg_gaisr", "b) buityje, vandenyje": "Isg_buit_vand",
                                    "c) transporto avarijose": "Isg_tr_av", "d) kitur": "Isg_buit_vand",
                                    "Evakuoti žmonės": "Evak_zm", "Melagingi pranešimai": "MP", "KITI IŠVYKIMAI": "KI",
                                     "Žuvę ne gaisre": "Zuve_negsr", "Traumuoti ne gaisre": "Trau_negsr" })
df1 = df1[['Data','Apskr', 'EkstSitu', 'EkstIvyk', 'GAISRAI',
               'GD_tr_av', 'GD_vand', 'GD_buit', 'GD_sp_trn', 'GD_ch_rad', 'GD_kiti',
               'MP', 'KI', 'Evak_zm', 'Isg_gaisr', 'Isg_buit_vand', 'Isg_tr_av', 'Isg_buit_vand',
               'Trau_gaisr', 'Trau_negsr', 'Zuve_gaisr', 'Zuve_negsr']]


############
#Paros gaisrų aprašymų lentelė
#df2 = pd.concat(dfs2, ignore_index=True)
############

In [5]:
df1.head()

,Data,Apskr,EkstSitu,EkstIvyk,GAISRAI,GD_tr_av,GD_vand,GD_buit,GD_sp_trn,GD_ch_rad,...,Isg_gaisr,Isg_buit_vand,Isg_buit_vand,Isg_tr_av,Isg_buit_vand,Isg_buit_vand,Trau_gaisr,Trau_negsr,Zuve_gaisr,Zuve_negsr
0,2015-03-20,Vilniaus apskritis,,1,85,,,,4,1,...,,,,,,,,,,
1,2015-03-20,Kauno apskritis,,,39,,,1,3,,...,2,,,,,,3,,,
2,2015-03-20,Klaipėdos apskritis,,,4,,,1,,,...,,,,,,,,,,
3,2015-03-20,Šiaulių apskritis,,,18,,,,,,...,,,,,,,,,,
4,2015-03-20,Panevėžio apskritis,,,26,2,,,,,...,,,,,,,,,,


In [6]:
#df1.to_csv('gaisrai_pgl_para.csv', index=False, header=True, encoding="cp1257")